In [ ]:
import numpy as np
from TMDP import TMDP
from algorithms import *
from model_functions import *
from algorithms import *
import matplotlib.pyplot as plt
import gymnasium as gym
from FrozenLake import *

#np.set_printoptions(precision=4)
import math
from utils import *

import torch
import mlflow
from CurriculumQ import CurriculumQ
import optuna

nrows = 80
nS = nrows**2
nA = 4
seed = get_current_seed()

gamma = .99
tau = .6
#tmdp = TMDP(river, xi, tau=tau, gamma=gamma, seed=seed)
#mdp = TMDP(river, xi, tau=0., gamma=gamma, seed=seed)

env = FrozenLakeEnv(is_slippery=False, seed=seed, 
                    desc=generate_random_map(nrows, seed=seed), 
                    reward_shape=True)#, render_mode="human")
xi = np.ones(env.nS) * 1/env.nS
tmdp = TMDP(env, xi, tau=tau, gamma=gamma, seed=seed)

In [ ]:
#res = bellman_optimal_q(tmdp.env.P_mat, tmdp.env.reward, tmdp.gamma)
#Q = res["Q"]
#V = compute_V_from_Q(Q, get_policy(Q))
#j_opt = compute_expected_j(V, tmdp.env.mu)
#print("optimal performance: ",j_opt)

In [ ]:
episodes = 2900000

model_lr = 0.15869281717397965

param_decay=True
lam = 1
batch_size = 32

eps_model = compute_eps_model(gamma, tau, episodes/10)

In [ ]:
curr_Q = CurriculumQ(tmdp)
tmdp.update_tau(tau) # Reset the teleport rate        

In [ ]:
print(curr_Q.Q)

In [ ]:
curr_Q.train(model_lr, batch_size=batch_size, 
                lam=lam, exp_rate=0.3,
                episodes=episodes,
                eps_model=eps_model,
                param_decay=param_decay)

In [ ]:
print(curr_Q.Q)

In [ ]:
"""
delta_pol = get_policy(Q)-get_policy(curr_Q.Q)
print(delta_pol)

r_s_a = compute_r_s_a(tmdp.env.P_mat, tmdp.env.reward)

V_Q = compute_V_from_Q(Q, get_policy(curr_Q.Q))

j_q_curr = compute_expected_j(V_Q, tmdp.env.mu)
j_opt = compute_expected_j(V, tmdp.env.mu)
print("optimal performance: ",j_opt)
print("Q curriculum performance: ",j_q_curr)
"""

In [ ]:
import matplotlib.pyplot as plt
exp_performances = curr_Q.exp_performances
# Generate recent 50 interval average
avg_performances = []
for idx in range(len(exp_performances)):
    avg_list = np.empty(shape=(1,), dtype=int)
    if idx < 50:
        avg_list = exp_performances[:idx+1]
    else:
        avg_list = exp_performances[idx-49:idx+1]
    avg_performances.append(np.average(avg_list))
# Plot
plt.plot(exp_performances)
#plt.plot(avg_performances)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

exp_performances = curr_Q.exp_performances
# Generate recent 50 interval average
avg_performances = []
std_errors = []

for idx in range(len(exp_performances)):
    if idx < 500:
        avg_list = exp_performances[:idx+1]
    else:
        avg_list = exp_performances[idx-499:idx+1]

    average = np.mean(avg_list)
    std_dev = np.std(avg_list)
    n_samples = len(avg_list)
    sem = std_dev / np.sqrt(n_samples)  # Standard Error of the Mean
    std_errors.append(sem)

    avg_performances.append(average)

# Compute the upper and lower bounds for the 95% confidence interval
ci = 1.96  # Z-score for 95% confidence
upper_bound = [avg + ci * sem for avg, sem in zip(avg_performances, std_errors)]
lower_bound = [avg - ci * sem for avg, sem in zip(avg_performances, std_errors)]

# Plot
plt.figure(figsize=(10, 5))
#plt.plot(exp_performances, label='Exp Performances')
plt.plot(avg_performances, label='Moving Average', color='r')
plt.fill_between(range(len(avg_performances)), lower_bound, upper_bound, color='r', alpha=0.2, label='95% Confidence Interval')
plt.legend()
plt.title('Experiment Performances with Moving Average and Confidence Interval')
plt.xlabel('Index')
plt.ylabel('Performance')
plt.show()


In [ ]:
pi = get_policy(curr_Q.Q)

In [ ]:

tmdp.update_tau(0.)
tmdp.reset()
done = False
step = 0
env.render_mode = "human"
while True:
    s = tmdp.env.s
    a = greedy(tmdp.env.s, pi, tmdp.env.allowed_actions[int(0)])
    s_prime, reward, flags, prob = tmdp.step(a)
    print("state {} action {} next state {} reward {}".format(s, a, s_prime, reward))
    if flags["done"]:
        tmdp.reset()
        break
    step +=1
    if step > max(100,int(nrows*2.5)):
        break


In [ ]:
print(curr_Q.Q)

In [ ]:
a = np.array([1, 2])
b = np.array([4, 1])
np.matmul(a, b)